# Question Answering – Fine-tuning – RoBERTa-base


Ce notebook entraîne un modèle de Question Answering extractif
sur les données SQuAD préprocessées.



## Objectifs

- Charger les données préprocessées
- Charger un modèle QA pré-entraîné
- Fine-tuner avec Trainer
- Sauvegarder le meilleur modèle


In [1]:
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
import numpy as np


c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Nombre de GPUs: {torch.cuda.device_count()}")
else:
    print("Pas de GPU disponible - utilisation du CPU")

CUDA disponible: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Nombre de GPUs: 1


## Chargement des données préprocessées

Les données ont été prétraitées dans le notebook précédent et sont
chargées directement depuis le disque.


In [3]:
tokenized_datasets = load_from_disk("outputs/tokenized_squad_small")

In [ ]:
# Supprimer token_type_ids (RoBERTa n'utilise pas cet input)
tokenized_datasets = tokenized_datasets.remove_columns("token_type_ids")

In [ ]:
# Inspecter la structure du dataset après suppression de token_type_ids
print("Dataset keys:", tokenized_datasets["train"].column_names)
sample = tokenized_datasets["train"][0]
print("\nSample keys:", sample.keys())
print("Sample shapes:")
for key, value in sample.items():
    if isinstance(value, list):
        print(f"  {key}: length={len(value)}")
    else:
        print(f"  {key}: {type(value)}")

Dataset keys: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']

Sample keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
Sample shapes:
  input_ids: length=384
  token_type_ids: length=384
  attention_mask: length=384
  start_positions: <class 'int'>
  end_positions: <class 'int'>

token_type_ids found in data:
  Values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
  Unique values: {0, 1}


## Data collator

Permet de créer les batchs correctement.


In [ ]:
# Simple data collator after token_type_ids removal
data_collator = DefaultDataCollator()

In [7]:
# Clear GPU memory before loading model
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None


## Tokenizer

Le tokenizer associé au modèle est chargé pour assurer la cohérence
du pipeline d’entraînement et d’inférence.


In [8]:
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


## Modèle

On charge un modèle compatible Question Answering.


## Note sur RoBERTa et token_type_ids

À la différence de BERT, RoBERTa n'utilise **pas** l'input `token_type_ids`.
Ce token a été supprimé du dataset avant l'entraînement pour éviter des incompatibilités.

La taille de RoBERTa-base est légèrement supérieure à DistilBERT, ce qui nécessite
une réduction du batch size (8 au lieu de 16) et l'utilisation de gradient accumulation
pour simuler un batch plus grand sans dépassement mémoire.

In [9]:
model_checkpoint = "roberta-base"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


Loading weights: 100%|██████████| 197/197 [00:00<00:00, 370.61it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForQuestionAnswering LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.bias              | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
qa_outputs.bias                 | MISSING    | 
qa_outputs.weight               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


## Note sur les couches apprises

Comme pour les autres modèles, certaines couches sont initialisées spécifiquement
pour la tâche de Question Answering (notamment `qa_outputs`) et sont apprises
durant le fine-tuning. C'est pourquoi certains poids sont signalés comme UNEXPECTED
ou MISSING au chargement du modèle pré-entraîné.

## Paramètres d’entraînement


In [ ]:
training_args = TrainingArguments(
    output_dir="outputs/checkpoints/roberta",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,  # Reduced from 16
    per_device_eval_batch_size=4,   # Reduced from 8
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="outputs/logs/roberta",
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2,  # Simulate larger batch size with less memory
    fp16=torch.cuda.is_available()  # Use mixed precision if GPU available
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


## Entraînement

On lance le fine-tuning avec Trainer.


In [ ]:
# Fixer la seed pour la reproductibilité
from transformers import set_seed
set_seed(42)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,4.754235,4.514801


Writing model shards: 100%|██████████| 1/1 [00:04<00:00,  4.34s/it]
There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.la

TrainOutput(global_step=125, training_loss=4.851259765625, metrics={'train_runtime': 257.6261, 'train_samples_per_second': 7.763, 'train_steps_per_second': 0.485, 'total_flos': 391945135104000.0, 'train_loss': 4.851259765625, 'epoch': 1.0})

In [ ]:
# Sauvegarder l'historique des métriques
import json

metrics_history = trainer.state.log_history
with open("outputs/roberta_metrics.json", "w") as f:
    json.dump(metrics_history, f, indent=2)
print("✓ Métriques sauvegardées dans outputs/roberta_metrics.json")

## Sauvegarde

In [15]:
trainer.save_model("outputs/checkpoints/roberta/final")
tokenizer.save_pretrained("outputs/checkpoints/roberta/final")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards: 100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


('outputs/checkpoints/roberta/final\\tokenizer_config.json',
 'outputs/checkpoints/roberta/final\\tokenizer.json')

## Conclusion

Le modèle a été fine-tuné et sauvegardé.
La prochaine étape consistera à évaluer la qualité (EM/F1) et la latence,
puis à comparer 3 modèles.
